<a href="https://colab.research.google.com/github/Ahnaf-045/TugasPemrogramanKomputer/blob/main/Tugas_Minggu_12_Kelompok_10_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas geopandas rasterio numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 87.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.sample import sample_gen
import numpy as np
from datetime import datetime
import os

def download_and_process_data():
    """
    Process geospatial data from GeoTIFF, GPKG, and CSV files.

    Note: Download the files from Google Drive first:
    1. GeoTIFF: https://drive.google.com/file/d/1-Zdq_ZXj4WoX5ubQnoHRjSlHfHrx07x0/view?usp=sharing
    2. GPKG: https://drive.google.com/file/d/1AQcHlqzmpQIukLyPIkUnb04-pbjoET_y/view?usp=sharing
    3. CSV: https://drive.google.com/file/d/1v8_2PwMxl1QGdcbC_RPgUGtYSZsmJDri/view?usp=sharing
    """

    # File paths (update these to match your downloaded files)
    geotiff_path = "your_geotiff_file.tif"  # Replace with actual filename
    gpkg_path = "your_survey_data.gpkg"     # Replace with actual filename
    csv_path = "perioda.csv"                # Replace with actual filename

    print("Starting geospatial data processing...")

    # Step 1: Load survey data from GPKG
    print("Loading survey data from GPKG...")
    try:
        survey_data = gpd.read_file(gpkg_path)
        print(f"Survey data loaded: {len(survey_data)} records")
        print("Columns:", survey_data.columns.tolist())
    except Exception as e:
        print(f"Error loading GPKG: {e}")
        return None

    # Step 2: Load period lookup table
    print("Loading period lookup table...")
    try:
        period_lookup = pd.read_csv(csv_path)
        print(f"Period lookup loaded: {len(period_lookup)} records")
        print("Columns:", period_lookup.columns.tolist())
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

    # Step 3: Filter and prepare survey data
    print("Filtering survey data...")

    # Extract coordinates (assuming geometry column exists)
    if 'geometry' in survey_data.columns:
        survey_data['lon'] = survey_data.geometry.x
        survey_data['lat'] = survey_data.geometry.y

    # Create filtered table with required columns
    # Adjust column names based on your actual data structure
    required_cols = ['fid', 'lon', 'lat', 'date', 'fase']

    # Check which columns exist and map them
    available_cols = survey_data.columns.tolist()
    print(f"Available columns: {available_cols}")

    # Create mapping for common column name variations
    column_mapping = {}
    for col in required_cols:
        if col in available_cols:
            column_mapping[col] = col
        elif col.upper() in available_cols:
            column_mapping[col] = col.upper()
        elif col.lower() in available_cols:
            column_mapping[col] = col.lower()
        elif col == 'fid' and 'FID' in available_cols:
            column_mapping[col] = 'FID'
        elif col == 'fid' and 'id' in available_cols:
            column_mapping[col] = 'id'
        elif col == 'date' and 'Date' in available_cols:
            column_mapping[col] = 'Date'
        elif col == 'fase' and 'phase' in available_cols:
            column_mapping[col] = 'phase'

    # Filter data with existing columns
    filtered_data = survey_data.copy()

    # Rename columns to standard names
    for new_name, old_name in column_mapping.items():
        if old_name in filtered_data.columns:
            filtered_data = filtered_data.rename(columns={old_name: new_name})

    # Select only required columns that exist
    existing_cols = [col for col in required_cols if col in filtered_data.columns]
    filtered_survey = filtered_data[existing_cols + ['geometry'] if 'geometry' in filtered_data.columns else existing_cols].copy()

    print(f"Filtered survey data: {len(filtered_survey)} records")
    print("Table 1 - Filtered Survey Data:")
    print(filtered_survey.head())

    # Step 4: Add period information
    print("Adding period information...")

    # Merge with period lookup table
    # Adjust merge column based on your data structure
    if 'date' in filtered_survey.columns and len(period_lookup.columns) >= 2:
        # Assuming period lookup has date and period columns
        period_col_names = period_lookup.columns.tolist()

        # Convert date columns to datetime for proper matching
        if 'date' in filtered_survey.columns:
            try:
                filtered_survey['date'] = pd.to_datetime(filtered_survey['date'])
            except:
                pass

        # Merge based on the first column of period lookup (assuming it's the key)
        merge_key = period_col_names[0]
        period_key = period_col_names[1] if len(period_col_names) > 1 else 'periode'

        try:
            # Try different merge strategies
            if merge_key in filtered_survey.columns:
                survey_with_period = filtered_survey.merge(
                    period_lookup,
                    left_on=merge_key,
                    right_on=period_col_names[0],
                    how='left'
                )
            else:
                # Create a simple period assignment based on date ranges or other logic
                survey_with_period = filtered_survey.copy()
                survey_with_period['periode'] = 1  # Default period

        except Exception as e:
            print(f"Merge error: {e}")
            survey_with_period = filtered_survey.copy()
            survey_with_period['periode'] = 1  # Default period
    else:
        survey_with_period = filtered_survey.copy()
        survey_with_period['periode'] = 1  # Default period

    print("Table 2 - Survey Data with Period:")
    print(survey_with_period.head())

    # Step 5: Extract raster values
    print("Extracting raster values...")

    try:
        with rasterio.open(geotiff_path) as src:
            print(f"Raster info: {src.count} bands, {src.width}x{src.height} pixels")

            # Prepare coordinates for sampling
            coords = [(lon, lat) for lon, lat in zip(survey_with_period['lon'], survey_with_period['lat'])]

            # Extract values for each point and each period
            final_results = []

            for idx, row in survey_with_period.iterrows():
                try:
                    coord = (row['lon'], row['lat'])
                    periode = int(row.get('periode', 1))

                    # Ensure period is within valid band range
                    if periode > src.count:
                        periode = 1

                    # Sample the raster at the point location for the specific band
                    sampled_values = list(sample_gen(src, [coord], indexes=[periode]))
                    p0_value = sampled_values[0][0] if sampled_values and len(sampled_values[0]) > 0 else np.nan

                    # Create result row
                    result_row = {
                        'fid': row.get('fid', idx),
                        'lon': row['lon'],
                        'lat': row['lat'],
                        'date': row.get('date', ''),
                        'fase': row.get('fase', ''),
                        'periode': periode,
                        'p0': p0_value
                    }

                    final_results.append(result_row)

                except Exception as e:
                    print(f"Error processing point {idx}: {e}")
                    continue

            # Create final DataFrame
            final_df = pd.DataFrame(final_results)

            print("Table 3 - Final Results with Raster Values:")
            print(final_df.head())
            print(f"\nFinal dataset: {len(final_df)} records")

            # Save results
            output_file = "processed_survey_data.csv"
            final_df.to_csv(output_file, index=False)
            print(f"Results saved to: {output_file}")

            return final_df

    except Exception as e:
        print(f"Error processing raster: {e}")
        return survey_with_period

def main():
    """
    Main function to run the data processing pipeline.
    """
    print("=== Geospatial Data Processing Pipeline ===")
    print()
    print("Before running this script, please:")
    print("1. Download the GeoTIFF file from Google Drive")
    print("2. Download the GPKG survey data file from Google Drive")
    print("3. Download the perioda.csv file from Google Drive")
    print("4. Update the file paths in the script")
    print()

    # Check if files exist
    files_to_check = ["your_geotiff_file.tif", "your_survey_data.gpkg", "perioda.csv"]
    missing_files = [f for f in files_to_check if not os.path.exists(f)]

    if missing_files:
        print("Missing files:")
        for f in missing_files:
            print(f"  - {f}")
        print("\nPlease download and update file paths before running.")
        return

    # Process the data
    result = download_and_process_data()

    if result is not None:
        print("\n=== Processing Complete ===")
        print("Three tables have been generated:")
        print("1. Filtered survey data (fid, lon, lat, date, fase)")
        print("2. Survey data with period (fid, lon, lat, date, fase, periode)")
        print("3. Final data with raster values (fid, lon, lat, date, fase, periode, p0)")
    else:
        print("Processing failed. Please check file paths and data format.")

if __name__ == "__main__":
    main()


# Alternative function for manual file processing
def process_with_custom_paths(geotiff_path, gpkg_path, csv_path):
    """
    Process data with custom file paths.

    Args:
        geotiff_path (str): Path to GeoTIFF file
        gpkg_path (str): Path to GPKG file
        csv_path (str): Path to CSV file
    """
    # Update the file paths in the global scope
    globals()['geotiff_path'] = geotiff_path
    globals()['gpkg_path'] = gpkg_path
    globals()['csv_path'] = csv_path

    return download_and_process_data()

=== Geospatial Data Processing Pipeline ===

Before running this script, please:
1. Download the GeoTIFF file from Google Drive
2. Download the GPKG survey data file from Google Drive
3. Download the perioda.csv file from Google Drive
4. Update the file paths in the script

Missing files:
  - your_geotiff_file.tif
  - your_survey_data.gpkg
  - perioda.csv

Please download and update file paths before running.


In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.sample import sample_gen
import numpy as np
from datetime import datetime
import os
import requests
import zipfile
from io import BytesIO

def download_from_gdrive(file_id, output_path):
    """
    Download file from Google Drive using file ID.

    Args:
        file_id (str): Google Drive file ID
        output_path (str): Local path to save the file
    """
    print(f"Downloading file ID: {file_id}")

    # Google Drive download URL
    url = f"https://drive.google.com/uc?export=download&id={file_id}"

    session = requests.Session()
    response = session.get(url, stream=True)

    # Handle Google Drive's virus scan warning for large files
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            params = {'id': file_id, 'confirm': value}
            response = session.get(url, params=params, stream=True)
            break

    # Save the file
    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(32768):
            if chunk:
                f.write(chunk)

    print(f"Downloaded: {output_path}")
    return output_path

def extract_file_id_from_url(drive_url):
    """
    Extract file ID from Google Drive sharing URL.

    Args:
        drive_url (str): Google Drive sharing URL

    Returns:
        str: File ID
    """
    if '/file/d/' in drive_url:
        return drive_url.split('/file/d/')[1].split('/')[0]
    elif 'id=' in drive_url:
        return drive_url.split('id=')[1].split('&')[0]
    else:
        raise ValueError("Cannot extract file ID from URL")

def download_and_process_data():
    """
    Download and process geospatial data from Google Drive links.
    """

    # Google Drive URLs (from your request)
    geotiff_url = "https://drive.google.com/file/d/1-Zdq_ZXj4WoX5ubQnoHRjSlHfHrx07x0/view?usp=sharing"
    gpkg_url = "https://drive.google.com/file/d/1AQcHlqzmpQIukLyPIkUnb04-pbjoET_y/view?usp=sharing"
    csv_url = "https://drive.google.com/file/d/1v8_2PwMxl1QGdcbC_RPgUGtYSZsmJDri/view?usp=sharing"

    # Extract file IDs
    geotiff_id = extract_file_id_from_url(geotiff_url)
    gpkg_id = extract_file_id_from_url(gpkg_url)
    csv_id = extract_file_id_from_url(csv_url)

    # Local file paths
    geotiff_path = "downloaded_geotiff.tif"
    gpkg_path = "downloaded_survey.gpkg"
    csv_path = "perioda.csv"

    print("Starting download and processing...")

    # Download files
    try:
        print("Downloading GeoTIFF...")
        download_from_gdrive(geotiff_id, geotiff_path)

        print("Downloading GPKG...")
        download_from_gdrive(gpkg_id, gpkg_path)

        print("Downloading CSV...")
        download_from_gdrive(csv_id, csv_path)

        print("All files downloaded successfully!")

    except Exception as e:
        print(f"Download error: {e}")
        print("Trying alternative download method...")

        # Alternative method using requests with different headers
        try:
            download_with_session(geotiff_url, geotiff_path)
            download_with_session(gpkg_url, gpkg_path)
            download_with_session(csv_url, csv_path)
        except Exception as e2:
            print(f"Alternative download also failed: {e2}")
            return None

    # Step 1: Load survey data from GPKG
    print("\nLoading survey data from GPKG...")
    try:
        survey_data = gpd.read_file(gpkg_path)
        print(f"Survey data loaded: {len(survey_data)} records")
        print("Columns:", survey_data.columns.tolist())
    except Exception as e:
        print(f"Error loading GPKG: {e}")
        return None

    # Step 2: Load period lookup table
    print("Loading period lookup table...")
    try:
        period_lookup = pd.read_csv(csv_path)
        print(f"Period lookup loaded: {len(period_lookup)} records")
        print("Columns:", period_lookup.columns.tolist())
        print("Sample data:")
        print(period_lookup.head())
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

    # Step 3: Filter and prepare survey data
    print("\nFiltering survey data...")

    # Extract coordinates (assuming geometry column exists)
    if 'geometry' in survey_data.columns:
        survey_data['lon'] = survey_data.geometry.x
        survey_data['lat'] = survey_data.geometry.y

    # Create filtered table with required columns
    required_cols = ['fid', 'lon', 'lat', 'date', 'fase']

    # Check which columns exist and map them
    available_cols = survey_data.columns.tolist()
    print(f"Available columns: {available_cols}")

    # Create mapping for common column name variations
    column_mapping = {}
    for col in required_cols:
        if col in available_cols:
            column_mapping[col] = col
        elif col.upper() in available_cols:
            column_mapping[col] = col.upper()
        elif col.lower() in available_cols:
            column_mapping[col] = col.lower()
        elif col == 'fid':
            # Try common ID column names
            for id_col in ['FID', 'id', 'ID', 'index', 'INDEX']:
                if id_col in available_cols:
                    column_mapping[col] = id_col
                    break
        elif col == 'date':
            # Try common date column names
            for date_col in ['Date', 'DATE', 'tanggal', 'waktu', 'time']:
                if date_col in available_cols:
                    column_mapping[col] = date_col
                    break
        elif col == 'fase':
            # Try common phase column names
            for phase_col in ['phase', 'Phase', 'FASE', 'PHASE']:
                if phase_col in available_cols:
                    column_mapping[col] = phase_col
                    break

    # Add missing columns with default values if not found
    filtered_data = survey_data.copy()

    # Add FID if not exists
    if 'fid' not in column_mapping and len(filtered_data) > 0:
        filtered_data['fid'] = range(1, len(filtered_data) + 1)
        column_mapping['fid'] = 'fid'

    # Rename columns to standard names
    for new_name, old_name in column_mapping.items():
        if old_name in filtered_data.columns and old_name != new_name:
            filtered_data = filtered_data.rename(columns={old_name: new_name})

    # Select required columns
    final_cols = []
    for col in required_cols:
        if col in filtered_data.columns:
            final_cols.append(col)
        else:
            # Add default column if missing
            if col == 'fase':
                filtered_data[col] = 'unknown'
            elif col == 'date':
                filtered_data[col] = datetime.now().strftime('%Y-%m-%d')
            final_cols.append(col)

    filtered_survey = filtered_data[final_cols].copy()

    print(f"\nTABEL 1 - Filtered survey data: {len(filtered_survey)} records")
    print("Columns:", filtered_survey.columns.tolist())
    print(filtered_survey.head())

    # Step 4: Add period information
    print("\nAdding period information...")

    # Check period lookup structure
    period_cols = period_lookup.columns.tolist()
    print(f"Period lookup columns: {period_cols}")

    # Try to merge with period lookup
    survey_with_period = filtered_survey.copy()

    # Simple period assignment based on available data
    if len(period_cols) >= 2:
        # Assume first column is key, second is period
        try:
            # Try different merge strategies
            merge_successful = False

            # Strategy 1: Direct merge on date
            if 'date' in filtered_survey.columns and period_cols[0].lower() in ['date', 'tanggal', 'waktu']:
                try:
                    survey_with_period = filtered_survey.merge(
                        period_lookup,
                        left_on='date',
                        right_on=period_cols[0],
                        how='left'
                    )
                    survey_with_period = survey_with_period.rename(columns={period_cols[1]: 'periode'})
                    merge_successful = True
                except:
                    pass

            # Strategy 2: Assign period based on row index or other logic
            if not merge_successful:
                # Create period assignment based on data distribution
                n_records = len(survey_with_period)
                n_periods = len(period_lookup)

                if n_periods > 0:
                    # Distribute records across periods
                    records_per_period = max(1, n_records // n_periods)
                    periods = []
                    for i in range(n_records):
                        period_idx = min(i // records_per_period, n_periods - 1)
                        periods.append(period_idx + 1)  # 1-based indexing
                    survey_with_period['periode'] = periods
                else:
                    survey_with_period['periode'] = 1
        except Exception as e:
            print(f"Period merge error: {e}")
            survey_with_period['periode'] = 1
    else:
        survey_with_period['periode'] = 1

    print(f"\nTABEL 2 - Survey data with period: {len(survey_with_period)} records")
    print("Columns:", survey_with_period.columns.tolist())
    print(survey_with_period.head())

    # Step 5: Extract raster values
    print("\nExtracting raster values...")

    try:
        with rasterio.open(geotiff_path) as src:
            print(f"Raster info: {src.count} bands, {src.width}x{src.height} pixels")
            print(f"CRS: {src.crs}")
            print(f"Bounds: {src.bounds}")

            # Extract values for each point
            final_results = []

            for idx, row in survey_with_period.iterrows():
                try:
                    lon = float(row['lon'])
                    lat = float(row['lat'])
                    coord = (lon, lat)
                    periode = int(row.get('periode', 1))

                    # Ensure period is within valid band range
                    if periode < 1:
                        periode = 1
                    elif periode > src.count:
                        periode = src.count

                    # Sample the raster at the point location
                    sampled_values = list(sample_gen(src, [coord], indexes=[periode]))

                    if sampled_values and len(sampled_values) > 0 and len(sampled_values[0]) > 0:
                        p0_value = float(sampled_values[0][0])
                        # Handle NoData values
                        if np.isnan(p0_value) or p0_value == src.nodata:
                            p0_value = None
                    else:
                        p0_value = None

                    # Create result row
                    result_row = {
                        'fid': row['fid'],
                        'lon': lon,
                        'lat': lat,
                        'date': row['date'],
                        'fase': row['fase'],
                        'periode': periode,
                        'p0': p0_value
                    }

                    final_results.append(result_row)

                except Exception as e:
                    print(f"Error processing point {idx}: {e}")
                    # Still add the row with null p0
                    result_row = {
                        'fid': row.get('fid', idx),
                        'lon': row.get('lon', 0),
                        'lat': row.get('lat', 0),
                        'date': row.get('date', ''),
                        'fase': row.get('fase', ''),
                        'periode': row.get('periode', 1),
                        'p0': None
                    }
                    final_results.append(result_row)

            # Create final DataFrame
            final_df = pd.DataFrame(final_results)

            print(f"\nTABEL 3 - Final results with raster values: {len(final_df)} records")
            print("Columns:", final_df.columns.tolist())
            print(final_df.head())

            # Statistics
            valid_p0 = final_df['p0'].notna().sum()
            print(f"\nStatistics:")
            print(f"Total records: {len(final_df)}")
            print(f"Valid p0 values: {valid_p0}")
            print(f"Null p0 values: {len(final_df) - valid_p0}")

            if valid_p0 > 0:
                print(f"P0 range: {final_df['p0'].min():.2f} to {final_df['p0'].max():.2f}")

            # Save results
            output_file = "processed_survey_data.csv"
            final_df.to_csv(output_file, index=False)
            print(f"\nResults saved to: {output_file}")

            # Clean up downloaded files
            cleanup_files = [geotiff_path, gpkg_path, csv_path]
            for file in cleanup_files:
                try:
                    if os.path.exists(file):
                        os.remove(file)
                        print(f"Cleaned up: {file}")
                except:
                    pass

            return final_df

    except Exception as e:
        print(f"Error processing raster: {e}")
        print("Returning data without raster values...")
        survey_with_period['p0'] = None
        return survey_with_period

def download_with_session(url, output_path):
    """
    Alternative download method using requests session.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }

    session = requests.Session()
    session.headers.update(headers)

    response = session.get(url, stream=True)
    response.raise_for_status()

    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)

def main():
    """
    Main function to run the data processing pipeline.
    """
    print("=== Geospatial Data Processing Pipeline ===")
    print("Downloading data from Google Drive and processing...")
    print()

    # Process the data
    result = download_and_process_data()

    if result is not None:
        print("\n=== Processing Complete ===")
        print("Tiga tabel telah berhasil dibuat:")
        print("1. Tabel 1: Data survey terfilter (fid, lon, lat, date, fase)")
        print("2. Tabel 2: Data survey dengan periode (fid, lon, lat, date, fase, periode)")
        print("3. Tabel 3: Data final dengan nilai raster (fid, lon, lat, date, fase, periode, p0)")
        print("\nFile hasil disimpan sebagai: processed_survey_data.csv")
    else:
        print("Processing gagal. Silakan periksa koneksi internet dan link Google Drive.")

if __name__ == "__main__":
    main()

=== Geospatial Data Processing Pipeline ===

Starting download and processing...
Downloaded: downloaded_geotiff.tif
Downloaded: downloaded_survey.gpkg
Downloaded: perioda.csv
All files downloaded successfully!

Loading survey data from GPKG...
Survey data loaded: 9401 records
Columns: ['lokasi', 'survei', 'sumber', 'bujur', 'lintang', 'kode', 'tanggal', 'fase', 'geometry']
Loading period lookup table...
Period lookup loaded: 31 records
Columns: ['Periode;Start Date;End Date']
Sample data:
  Periode;Start Date;End Date
0     1;2024-01-01;2024-01-13
1     2;2024-01-13;2024-01-25
2     3;2024-01-25;2024-02-06
3     4;2024-02-06;2024-02-18
4     5;2024-02-18;2024-03-01

Filtering survey data...
Available columns: ['lokasi', 'survei', 'sumber', 'bujur', 'lintang', 'kode', 'tanggal', 'fase', 'geometry', 'lon', 'lat']

TABEL 1 - Filtered survey data: 9401 records
Columns: ['fid', 'lon', 'lat', 'date', 'fase']
   fid         lon       lat       date  fase
0    1  107.463959 -6.260439  3/18/202